In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
sns.set(style='whitegrid')
file_path = "superstore.csv"

# Load Data
df = pd.read_csv(file_path)

print("Columns: ", df.columns.to_list())
print("\nShape: ", df.shape)
print("\nInfo: ")
print(df.info())

Columns:  ['Category', 'City', 'Country', 'Customer.ID', 'Customer.Name', 'Discount', 'Market', '记录数', 'Order.Date', 'Order.ID', 'Order.Priority', 'Product.ID', 'Product.Name', 'Profit', 'Quantity', 'Region', 'Row.ID', 'Sales', 'Segment', 'Ship.Date', 'Ship.Mode', 'Shipping.Cost', 'State', 'Sub.Category', 'Year', 'Market2', 'weeknum']

Shape:  (51290, 27)

Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51290 non-null  object 
 1   City            51290 non-null  object 
 2   Country         51290 non-null  object 
 3   Customer.ID     51290 non-null  object 
 4   Customer.Name   51290 non-null  object 
 5   Discount        51290 non-null  float64
 6   Market          51290 non-null  object 
 7   记录数             51290 non-null  int64  
 8   Order.Date      51290 non-null  object 
 9   Order.ID        51290 non

# Clean & Prepare Data

In [30]:
# Drop unknown column drop 
cols_to_drop = ["记录数"]
df = df.drop(columns=cols_to_drop, errors="ignore")

# 3) Date column parse
df["Order.Date"] = pd.to_datetime(df["Order.Date"])
df["Ship.Date"] = pd.to_datetime(df["Ship.Date"])

# Quick NA & describe
print("\nMissing values:\n", df.isnull().sum())
print("\nDescribe numeric:\n", df.describe())


Missing values:
 Category          0
City              0
Country           0
Customer.ID       0
Customer.Name     0
Discount          0
Market            0
Order.Date        0
Order.ID          0
Order.Priority    0
Product.ID        0
Product.Name      0
Profit            0
Quantity          0
Region            0
Row.ID            0
Sales             0
Segment           0
Ship.Date         0
Ship.Mode         0
Shipping.Cost     0
State             0
Sub.Category      0
Year              0
Market2           0
weeknum           0
dtype: int64

Describe numeric:
            Discount                     Order.Date        Profit  \
count  51290.000000                          51290  51290.000000   
mean       0.142908  2013-05-11 21:26:49.155780864     28.610982   
min        0.000000            2011-01-01 00:00:00  -6599.978000   
25%        0.000000            2012-06-19 00:00:00      0.000000   
50%        0.000000            2013-07-08 00:00:00      9.240000   
75%        0.200000  

# Visualize

In [31]:
# ! pip3 install plotly
# ! pip3 install -U kaleido

In [32]:
import plotly.express as px
import plotly.graph_objects as go

# Dark layout
def dark_layout(fig, title=None):
    fig.update_layout(
        template="plotly_dark",
        title=title,
        paper_bgcolor="#111827",
        plot_bgcolor="#111827",
        font=dict(color="white")
    )
    return fig

# Save figures
def save_fig(fig, name):
    fig.write_image(f"charts/static/{name}.png")
    fig.write_html(f"charts/interactive/{name}.html")

# 1. Sales vs Category  (bar)
cat_sales = df.groupby("Category", as_index=False)["Sales"].sum()

fig1 = px.bar(
    cat_sales,
    x="Category",
    y="Sales",
    color="Category",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig1 = dark_layout(fig1, "Sales by Product Category")
save_fig(fig1, "sales_by_category")
fig1.show()

# 2. Monthly Sales Trend  (line)
monthly_sales = df.groupby(df["Order.Date"].dt.to_period("M"))["Sales"].sum()
monthly_sales.index = monthly_sales.index.to_timestamp()
monthly_df = monthly_sales.reset_index()
monthly_df.columns = ["Month", "Sales"]

fig2 = px.line(
    monthly_df,
    x="Month",
    y="Sales",
    markers=True,
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig2 = dark_layout(fig2, "Monthly Sales Trend")
save_fig(fig2, "monthly_sales_trend")
fig2.show()

# 3. Discount vs Profit  (scatter)
fig3 = px.scatter(
    df,
    x="Discount",
    y="Profit",
    color="Discount",
    opacity=0.7,
    color_continuous_scale=px.colors.sequential.Peach,
)
fig3.update_traces(marker=dict(line=dict(width=0.3, color="white")))
fig3 = dark_layout(fig3, "Discount vs Profit")
save_fig(fig3, "discount_vs_profit")
fig3.show()

# 4. Sales by Region  (bar)
region_sales = df.groupby("Region", as_index=False)["Sales"].sum()

fig4 = px.bar(
    region_sales,
    x="Region",
    y="Sales",
    color="Region",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig4 = dark_layout(fig4, "Sales by Region")
save_fig(fig4, "sales_by_region")
fig4.show()

# 5. Profit by Region  (bar)
region_profit = df.groupby("Region", as_index=False)["Profit"].sum()

fig5 = px.bar(
    region_profit,
    x="Region",
    y="Profit",
    color="Region",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig5 = dark_layout(fig5, "Profit by Region")
save_fig(fig5, "profit_by_region")
fig5.show()

# 6. Top 10 Profitable Products  (horizontal bar)
top_products = (
    df.groupby("Product.Name", as_index=False)["Profit"]
      .sum()
      .nlargest(10, "Profit")
)

fig6 = px.bar(
    top_products,
    x="Profit",
    y="Product.Name",
    orientation="h",
    color="Profit",
    color_continuous_scale=px.colors.sequential.Peach,
)
fig6 = dark_layout(fig6, "Top 10 Profitable Products")
save_fig(fig6, "top_10_profitable_products")
fig6.show()

# 7a. Sales by Customer Segment  (bar)
seg_sales = df.groupby("Segment", as_index=False)["Sales"].sum()

fig7a = px.bar(
    seg_sales,
    x="Segment",
    y="Sales",
    color="Segment",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig7a = dark_layout(fig7a, "Sales by Customer Segment")
save_fig(fig7a, "sales_by_segment")
fig7a.show()

# 7b. Profit by Customer Segment  (bar)
seg_profit = df.groupby("Segment", as_index=False)["Profit"].sum()

fig7b = px.bar(
    seg_profit,
    x="Segment",
    y="Profit",
    color="Segment",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig7b = dark_layout(fig7b, "Profit by Customer Segment")
save_fig(fig7b, "profit_by_segment")
fig7b.show()

# 8. Shipping Cost vs Profit  (scatter)
fig8 = px.scatter(
    df,
    x="Shipping.Cost",
    y="Profit",
    color="Shipping.Cost",
    opacity=0.7,
    color_continuous_scale=px.colors.sequential.Blues,
)
fig8.update_traces(marker=dict(line=dict(width=0.3, color="white")))
fig8 = dark_layout(fig8, "Shipping Cost vs Profit")
save_fig(fig8, "shipping_cost_vs_profit")
fig8.show()

# 9. Delivery Time vs Profit  (box)
df["delivery_days"] = (df["Ship.Date"] - df["Order.Date"]).dt.days

fig9 = px.box(
    df,
    x="delivery_days",
    y="Profit",
    color="delivery_days",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig9 = dark_layout(fig9, "Delivery Time vs Profit")
save_fig(fig9, "delivery_time_vs_profit")
fig9.show()

# 10. Loss-Making Orders  (table)
loss_orders = df[df["Profit"] < 0].copy()

table_cols = ["Order.ID", "Product.Name", "Region", "Discount", "Profit"]
table_cols = [c for c in table_cols if c in loss_orders.columns]

fig10 = go.Figure(
    data=[go.Table(
        header=dict(values=table_cols, fill_color="#4B5563", font=dict(color="white")),
        cells=dict(values=[loss_orders[c].head(10) for c in table_cols],
                   fill_color="#111827", font=dict(color="white"))
    )]
)
fig10 = dark_layout(fig10, "Sample of Loss-Making Orders")
save_fig(fig10, "loss_making_orders_sample")
fig10.show()

# 11. Yearly Sales  (bar)
year_sales = df.groupby("Year", as_index=False)["Sales"].sum()

fig11 = px.bar(
    year_sales,
    x="Year",
    y="Sales",
    color="Year",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig11 = dark_layout(fig11, "Yearly Sales")
save_fig(fig11, "yearly_sales")
fig11.show()
